In [ ]:
!pip install -q peft datasets
!pip install -q accelerate bitsandbytes

In [ ]:
!pip install -q transformers

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup ,TrainingArguments
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import DataCollatorWithPadding

In [ ]:
device = "cuda"
model_name_or_path = "databricks/dolly-v2-3b"
tokenizer_name_or_path = "databricks/dolly-v2-3b"

text_column = "prompt"
label_column = "label"

In [ ]:
from datasets import load_from_disk

# Specify the directory where the dataset is saved
saved_directory = "/content/drive/MyDrive/Colab Notebooks/Table to insights/Data/Analytical Datset"

# Load the dataset from the specified directory
dataset = load_from_disk(saved_directory)


In [ ]:
dataset['train'].features

{'table': [{'cpu_usage': Value(dtype='float64', id=None),
   'date': Value(dtype='string', id=None),
   'diskio_usage': Value(dtype='float64', id=None),
   'ram_usage': Value(dtype='float64', id=None)}],
 'prompt': Value(dtype='string', id=None),
 'label': Value(dtype='string', id=None)}

# Trainer API

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(tokenizer_name_or_path)

In [ ]:
def preprocess_data(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs,padding="max_length",max_length = 1200,truncation=True, return_tensors="pt")
    labels = tokenizer(targets, padding="max_length", max_length = 1200,truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_data,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/516 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/129 [00:00<?, ? examples/s]

In [ ]:
processed_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 516
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 129
    })
})

In [ ]:
train_dataset = processed_datasets['train']
validation_dataset = processed_datasets['validation']

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,load_in_8bit=True,device_map='auto')

In [ ]:
from peft import prepare_model_for_int8_training

In [ ]:
model.train()
model.resize_token_embeddings(len(tokenizer))
def create_peft_config(model):

    peft_config = PrefixTuningConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        num_virtual_tokens=20
    )

    # prepare int-8 model for training
    # model = prepare_model_for_int8_training(model)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

model, peft_config = create_peft_config(model)

trainable params: 3,276,800 || all params: 2,778,362,880 || trainable%: 0.11793995750475907


In [ ]:
output_dir = "/content/drive/MyDrive/Colab Notebooks/Table to insights/dolly"

In [ ]:
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=10,
    save_strategy="no",
    optim="adamw_hf",
    learning_rate = 1e-4,
    num_train_epochs=1,
    gradient_accumulation_steps= 1,
    per_device_train_batch_size= 1,
    gradient_checkpointing=False
)

# Auto trainer

In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=validation_dataset,
        # data_collator=default_data_collator,
    )

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss
10,10.828100
20,10.523400
30,10.507800
40,10.686700
50,10.401600
60,10.541400
70,10.481200
80,10.264800
90,10.185900
100,10.185200


TrainOutput(global_step=516, training_loss=9.732936652131784, metrics={'train_runtime': 703.3091, 'train_samples_per_second': 0.734, 'train_steps_per_second': 0.734, 'total_flos': 9831791149056000.0, 'train_loss': 9.732936652131784, 'epoch': 1.0})

In [ ]:
trainer.save_model("/content/drive/MyDrive/Colab Notebooks/Table to insights/dolly/prefix_tuned_model_1200")